In [ ]:
# ==================== 导入必要的库 ====================
# collections: 提供专门的容器数据类型，如Counter用于计数
# re: 正则表达式模块，用于文本处理
# d2l: Dive into Deep Learning工具库，提供深度学习相关的辅助函数
import collections  # 导入collections模块，用于词频统计
import re  # 导入正则表达式模块，用于文本清理
from d2l import torch as d2l  # 从d2l导入torch相关工具

In [ ]:
# ==================== 下载和读取时间机器数据集 ====================
# 什么是时间机器数据集？
# - 《时间机器》小说全文
# - 用于自然语言处理任务的经典数据集
# - 包含大量英文文本，用于训练语言模型

# 将数据集添加到d2l的数据中心
# 'time_machine': 数据集名称
# d2l.DATA_URL + 'timemachine.txt': 下载URL
# '090b5e7e70c295757f55df93cb0a180b9691891a': 文件哈希值，用于验证完整性
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')  # 注册数据集到d2l数据中心

def read_time_machine():  # 定义读取时间机器数据集的函数
    """
    加载时间机器数据集到文本行列表
    
    处理步骤：
    1. 下载数据集文件
    2. 读取所有行
    3. 使用正则表达式清理文本：
       - [^A-Za-z]+: 匹配非字母字符
       - 替换为空格
       - 转换为小写
       - 去除首尾空格
    
    返回：
        清理后的文本行列表
    """
    with open(d2l.download('time_machine'), 'r') as f:  # 下载并打开文件
        lines = f.readlines()  # 读取所有行到列表
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]  # 清理每一行：替换非字母为空格，转小写，去除首尾空格

# 读取数据集
lines = read_time_machine()  # 调用函数读取数据集

# 输出基本信息
print(f'# 句子数: {len(lines)}')  # 打印句子数量
print(lines[0])  # 打印第一行
print(lines[10])  # 打印第11行

# 句子数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [ ]:
# ==================== 文本分词函数 ====================
# 什么是分词（tokenization）？
# - 将连续的文本分割成有意义的单元（词元）
# - 词元可以是单词（word）或字符（char）
# - 这是NLP预处理的基础步骤

def tokenize(lines, token='word'):  # 定义分词函数，参数：文本行列表，分词类型（默认单词）
    """
    将文本行拆分为单词或字符词元
    
    参数：
        lines: 文本行列表
        token: 分词类型，'word'（单词）或'char'（字符）
    
    返回：
        分词后的词元列表，每个元素是一个词元列表
    
    示例：
        输入：["hello world", "how are you"]
        输出（word）：[["hello", "world"], ["how", "are", "you"]]
        输出（char）：[["h","e","l","l","o"," ","w","o","r","l","d"], ...]
    """
    if token == 'word':  # 如果分词类型是单词
        return [line.split() for line in lines]  # 按空格分割每一行，返回单词列表
    elif token == 'char':  # 如果分词类型是字符
        return [list(line) for line in lines]  # 将每一行转换为字符列表
    else:  # 否则
        print('错误：未知词元类型：' + token)  # 打印错误信息
        
# 对时间机器数据集进行分词（按单词）
tokens = tokenize(lines)  # 调用分词函数，默认按单词分词

# 显示前11行的分词结果
for i in range(11):  # 循环前11次
    print(tokens[i])  # 打印第i行的分词结果

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [ ]:
# ==================== 词汇表（Vocab）类 ====================
# 什么是词汇表？
# - 将词元映射到数字索引的字典
# - 用于将文本转换为模型可以处理的数字序列
# - 通常按词频排序，最常见的词有较小的索引

class Vocab:  # 定义词汇表类
    """
    文本词表：词元到索引的映射
    
    主要功能：
    - 统计词频，按频率排序
    - 创建词元到索引的映射
    - 支持未知词元处理
    - 提供索引到词元的反向映射
    """
    
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):  # 初始化方法，参数：词元列表，最小频率，保留词元
        """
        初始化词汇表
        
        参数：
            tokens: 词元列表，可以是1D或2D列表
            min_freq: 最小词频阈值，低于此频率的词元将被过滤
            reserved_tokens: 保留词元列表，如特殊符号<pad>、<bos>等
        
        处理流程：
        1. 统计所有词元的频率
        2. 按频率降序排序
        3. 过滤低频词元
        4. 创建索引映射（未知词元索引为0）
        """
        if tokens is None:  # 如果tokens为None
            tokens = []  # 设置为空列表
        if reserved_tokens is None:  # 如果reserved_tokens为None
            reserved_tokens = []  # 设置为空列表
            
        # 按出现频率排序词元
        counter = count_corpus(tokens)  # 统计词频
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],  # 按频率降序排序
                                   reverse=True)
        
        # 未知词元的索引为0
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens  # 设置未知词元索引，初始化唯一词元列表
        
        # 添加满足频率要求的词元
        uniq_tokens += [  # 扩展唯一词元列表
            token for token, freq in self._token_freqs  # 遍历词频对
            if freq >= min_freq and token not in uniq_tokens  # 如果频率足够且不在列表中
        ]
        
        # 创建双向映射
        self.index_to_token, self.token_to_idx = [], dict()  # 初始化索引到词元和词元到索引的映射
        for token in uniq_tokens:  # 遍历唯一词元
            self.index_to_token.append(token)  # 添加到索引到词元列表
            self.token_to_idx[token] = len(self.index_to_token) - 1  # 设置词元到索引映射

    def __len__(self):  # 定义长度方法
        """返回词汇表大小"""
        return len(self.index_to_token)  # 返回索引到词元列表的长度

    def __getitem__(self, tokens):  # 定义索引方法，支持词元到索引转换
        """
        将词元转换为索引
        
        参数：
            tokens: 单个词元或词元列表
        
        返回：
            对应的索引或索引列表
        """
        if not isinstance(tokens, (list, tuple)):  # 如果不是列表或元组（单个词元）
            return self.token_to_idx.get(tokens, self.unk)  # 返回索引，未知词元返回unk索引
        return [self.__getitem__(token) for token in tokens]  # 递归转换列表中的每个词元

    def to_tokens(self, indices):  # 定义反向转换方法
        """
        将索引转换为词元
        
        参数：
            indices: 单个索引或索引列表
        
        返回：
            对应的词元或词元列表
        """
        if not isinstance(indices, (list, tuple)):  # 如果不是列表或元组（单个索引）
            return self.index_to_token[indices]  # 返回对应的词元
        return [self.index_to_token[index] for index in indices]  # 返回索引列表对应的词元列表

def count_corpus(tokens):  #@save  # 定义词频统计函数
    """
    统计词元的频率
    
    参数：
        tokens: 词元列表，可以是1D或2D列表
    
    返回：
        词元频率的Counter对象
    
    处理逻辑：
    - 如果输入是2D列表（多行文本），先展平为一维
    - 使用collections.Counter统计每个词元的出现次数
    """
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):  # 如果tokens为空或第一个元素是列表（2D）
        tokens = [token for line in tokens for token in line]  # 展平为1D列表
    return collections.Counter(tokens)  # 返回词频统计结果

In [ ]:
# ==================== 创建词汇表实例 ====================
# 使用分词后的词元创建词汇表
# 这将建立词元到数字索引的映射关系
vocab = Vocab(tokens)  # 使用分词结果创建词汇表实例

# 显示词汇表前10个词元及其索引
# 观察：
# - '<unk>': 索引0，未知词元
# - 最常见的词元有较小的索引
print(list(vocab.token_to_idx.items())[:10])  # 打印词汇表前10项

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [ ]:
# ==================== 文本到索引的转换示例 ====================
# 演示如何将原始文本转换为数字索引序列
# 这展示了NLP预处理的完整流程

for i in [0, 10]:  # 遍历第0行和第10行
    print('文本:', tokens[i])  # 打印原始文本
    
    print('索引:', vocab[tokens[i]])  # 打印对应的索引序列

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [ ]:
# ==================== 加载时光机器语料库 ====================
# 什么是语料库？
# - 经过预处理的文本数据集
# - 通常是词元索引的序列
# - 用于训练语言模型或进行其他NLP任务

def load_corpus_time_machine(max_tokens=-1):  # 定义加载语料库函数，参数：最大词元数
    """
    返回时光机器数据集的词元索引列表和词表
    
    参数：
        max_tokens: 最大词元数量，-1表示使用全部词元
    
    返回：
        corpus: 词元索引的列表（一维）
        vocab: 词汇表对象
    
    处理流程：
    1. 读取原始文本行
    2. 按字符分词（而不是单词）
    3. 创建词汇表
    4. 将所有词元转换为索引
    5. 可选：截取前max_tokens个词元
    """
    lines = read_time_machine()  # 读取文本行
    tokens = tokenize(lines, 'char')  # 按字符分词
    vocab = Vocab(tokens)  # 创建词汇表
    
    # 将文本转换为词元的索引表示
    corpus = [vocab[token] for line in tokens for token in line]  # 展平并转换为索引
    
    if max_tokens > 0:  # 如果指定了最大词元数
        corpus = corpus[:max_tokens]  # 截取前max_tokens个词元
    
    return corpus, vocab  # 返回语料库和词汇表

# 加载完整的语料库
corpus, vocab = load_corpus_time_machine()  # 调用函数加载语料库

# 输出语料库的基本信息
print('# 词元数:', len(corpus))  # 打印词元总数
print('# 词表大小:', len(vocab))  # 打印词汇表大小

# 词元数: 170580
# 词表大小: 28
